# Einführung in die Programmiersprache Python und wissenschaftlich-technische Anwendungsbeispiele - Teil II
## Inhalt
* Coolprop
* Einführung in die objektorientierte Modellierung (OOM)

## Coolprop
### Schnittstelle
+ High-level interface (CoolProp.CoolProp)
  - einfache Anwendung (für die meisten Fälle ausreichend)
  - leicht verständlich
+ Low-level interface (CoolProp)
  - fortgeschrittene Anwendung
  - schneller

### Häufig verwendete Funktionen in der High-level interface (CoolProp.CoolProp)
+ **PropsSI()**: Berechnung der Stoffeigenschaft eines Fluid (anhand zwei gegebener Zustandsgrößen)
+ **PhaseSI()**: Ermittlung der Phase des Fluids für den gegebenen Zustandspunkt
+ **get_fluid_param_string()**: Abrufen der Informationen eines Fluidstrings, z.B. chemische Formel des Fluids
+ **set_reference_state()**: Änderung der Referenzpunkt für die Berechnung der Enthalpie / Entropie

In [7]:
# Importieren von Paketen und Funktionen
import CoolProp.CoolProp as CP
from CoolProp.CoolProp import PropsSI
import scipy.optimize as opt
import numpy as np

ModuleNotFoundError: No module named 'CoolProp'

+ Berechnung des Sättigungsdampfdrucks (auf Siedelinie)

In [2]:
p_s = PropsSI('P', 'T', 25+273.15, 'Q', 0, "CO2")
print(p_s, "Pa")

6434244.250641562 Pa


+ Berechnung der Enthalpie

In [3]:
h = PropsSI('H', 'P', 1e5, 'T', 20+273.15, "air")
print(h, "J/kg")

419408.0704572205 J/kg


#### Hinweise:
+ Die Parameter z.B. P, T hier müssen in Großschreibung angegeben werden
+ eine Liste der Parameter siehe http://www.coolprop.org/coolprop/HighLevelAPI.html#fluid-information
+ Mehr Beispiele in der Übung

## Einführung in die objektorientierte Modellierung (OOM) in Python
### Objekte und Klassen
+ Ein Objekt enhält
  - Eigenschaften/Attribute (Variable)
  - Methoden/Funktionalitäten (Funktionen)
+ Klasse
  - Muster / Vorlage zum Erstellen von ähnlichen Objekten mit gleichen Attributen und Methoden
  
### Merkmale und wesentliche Vorteile
+ Datenkapselung: Verbergen von Daten oder Informationen vor dem Zugriff von außen.
+ Vererbung: Erstellung der Klassen, die auf vorhandenen Klassen aufbauen.
+ Polymorphie (Vielgestaltigkeit): Methode mit identischen Signaturen kann in verschiedenen Klassen unterschiedlich implementiert werden.
+ strukturiert, übersichtlich, flexibel

### Deklaration und Nutzung von Klassen in Python
+ **class** Schlüsselwort für Klassedeklaration
+ Abgrenzung von Blöcken mit Einrückung
+ **def** Schlüsselwort für Deklaration der Klassenmethoden (Funktionen)
  - __init__() Methode für Vorgabewerten / Initialisierung, kein Rückgabewert
  - Methoden können Parameter / Argumente in () enthalten. self ist das erste Argument einer Methode. Dann folgen die sonstigen Parameter.
+ **self.** Instanz der Klasse
  - self.Name_des_Attributes für Aufruf eines Attributes innerhalb der Klasse
  - self.Name_der_Methode() für Aufruf einer Methode innerhalb der Klasse
+ Objektname.Attribute / Objektname.Methode() für Aufruf der Attribute und Methoden einer Objekt

In [4]:
class Ventilator: 
    # enthält 3 Attribute: n, n_max, Name (als optionale Eingabe)
    # und 2 Methode: start(), stop()
    def __init__(self, max_Drehzahl, Name = None): # Vorgabewerte
        self.n_max = max_Drehzahl # Attribute 1
        self.n = 0 # Attribute 2
        self.Name = Name # Attribute 3
        
    def start(self): # Methode 1
        print('Ventilator {} wird eingeschaltet'.format(self.Name))
        self.n = self.n_max
    def stop(self): # Methode 2
        print('Ventilator {} wird ausgeschaltet'.format(self.Name))
        self.n = 0
    # ... ggf. noch weitere Methoden

In [5]:
Vent1 = Ventilator(1000, Name = 'A')
Vent1.start()
print(Vent1.n, 'rpm')
Vent1.stop()
print(Vent1.n, 'rpm')

Ventilator A wird eingeschaltet
1000 rpm
Ventilator A wird ausgeschaltet
0 rpm


### Vererbung
+ Abgeleitete Klasse (Unterklasse, erbende Klasse), die die Attribute und Methoden von der Basisklasse (Oberklasse, vererbende Klasse) enthält
+ Deklaration einer abgeleiteten Klasse: class Name_der_abgleiteten_Klasse(Name_der_Basisklasse):
+ Überschreiben von Methoden möglich


In [6]:
class Besserer_Ventilator(Ventilator):
    # die Klasse vererbt die Attribute `n`, `n_max`, `Name` 
    # und Methoden `start()`, `stop()` von der Basisklasse "Ventilator"
    
    # Deklaration und Definition weiterer Methoden
    def Drehzahl_einstellen(self, n):
        if self.n == 0: # wenn der Ventilator nicht eingescahltet ist
            print('Ventilator {} wird eingeschaltet'.format(self.Name))
        self.n = min(n, self.n_max)
    # ... ggf. noch weitere Methoden 

In [7]:
Vent2 = Besserer_Ventilator(1000, Name = 'B')
Vent2.Drehzahl_einstellen(600)
print(Vent2.n, 'rpm')
Vent2.stop()
print(Vent2.n, 'rpm')

Ventilator B wird eingeschaltet
600 rpm
Ventilator B wird ausgeschaltet
0 rpm


## Anwendungsbeispiele
### Ein einfaches Verdichter-Modell
##### **Bekannte Größe**:
V_Hub, η_is, η_el, n_norm, λ
##### **Betriebsparameter**:
T_1, p_1, p_2, n
##### **Die Gleichungen zur Berechnung des Fördermassenstroms und der Leistungen**:
h_2 = h_1 + (h_2_is − h_1)/ η_is

m = λ ⋅ V_Hub / v_1 ⋅ n / n_norm

P_i = m ⋅ (h_2 − h_1)

P_el = P_i / η_el

#### Modellierung als Funktion

In [8]:
def fun_Verdichter(fluid = None, 
                 Fördervolumen = 0, # m3/h
                 isentroper_Guetegrad = 1,
                 elektrischer_Wirkungsgrad = 1,
                 Liefergrad = 1,
                 Betriebsparameter = {}, # z.B. Eingabe p_1, T_1, p_2, n
              ): 
    h_1 = PropsSI('H', 'P', Betriebsparameter['p_1'], 'T', Betriebsparameter['T_1'], fluid)
    s_1 = PropsSI('S', 'P', Betriebsparameter['p_1'], 'T', Betriebsparameter['T_1'], fluid)
    v_1 = 1/PropsSI('D', 'P', Betriebsparameter['p_1'], 'T', Betriebsparameter['T_1'], fluid)
    h_2_is = PropsSI('H', 'S', s_1, 'P', Betriebsparameter['p_2'], fluid)
    h_2 = h_1 + (h_2_is - h_1) / isentroper_Guetegrad
    T_2 = PropsSI('T', 'P', Betriebsparameter['p_2'], 'H', h_2, fluid)
    m_dot = Liefergrad * Fördervolumen / 50* Betriebsparameter['n'] /3600 / v_1 # Fördermassenstrom in kg3/h
    P_i = m_dot * (h_2 - h_1) #innere Leistung
    P_el = P_i / elektrischer_Wirkungsgrad #elektrische Leistung
    return m_dot, P_i, P_el

In [9]:
BP1 = {
            'p_1' : 45e5,
            'T_1' : 20+273.15,
            'p_2' : 90e5,
            'n' : 50
      }

print(fun_Verdichter(fluid = 'CO2', Fördervolumen = 15, isentroper_Guetegrad = 0.7, elektrischer_Wirkungsgrad = 0.95,
                Liefergrad = 0.8,                 
                Betriebsparameter = BP1))

(0.38961248651570357, 16087.820158068509, 16934.547534808957)


#### Modellierung als Klasse (OOM)

In [10]:
class Verdichter():
    def __init__(self, fluid = None, 
                 Fördervolumen = 0, # m3/h
                 isentroper_Guetegrad = 1,
                 elektrischer_Wirkungsgrad = 1,
                 Liefergrad = 1,
                 Betriebsparameter = {}, # z.B. Eingabe p_1, T_1, p_2, n
                 ):
        self.fluid = fluid
        self.eta_is = isentroper_Guetegrad
        self.eta_el = elektrischer_Wirkungsgrad
        self.lbd = Liefergrad
        self.V_dot_th_norm = Fördervolumen
        self.P_i = 0. # innere Leistung
        self.m_dot = 0. # Massenstrom
        self.BP = self.__BP_norm()
        self.update_BP(Betriebsparameter) 
    
    def update_BP(self, BP): # neue Betriebsparameter eingeben
        self.BP.update(BP)
        
    def __BP_norm(self): 
        BP = {
            'p_1' : 0, # Eintrittsdruck
            'T_1' : 0, # Eintrittstemperatur
            'p_2' : 0, # Austrittsdruck
            'n' : 50, # in Hz Drehzahl
        }
        return BP 

In [11]:
class Verdichter(Verdichter): 
    # Fortsetzung der vorherigen Zelle, 
    #nur zu Darstellungszwecken, nicht für normale Arbeiten nutzen
    
    @property # automatisch aktualiesiert
    def V_dot_th(self): #theoretische Volumenstrom in m3/h
        return self.V_dot_th_norm * self.BP['n'] / self.__BP_norm()['n']
    
    def berechnen(self):
        # berechnung self.h_o, self.P und self.V_th 
        self.h_1 = PropsSI('H', 'P', self.BP['p_1'], 'T', self.BP['T_1'], self.fluid)
        s_1 = PropsSI('S', 'P', self.BP['p_1'], 'T', self.BP['T_1'], self.fluid)
        v_1 = 1/PropsSI('D', 'P', self.BP['p_1'], 'T', self.BP['T_1'], self.fluid)
        h_2_is = PropsSI('H', 'S', s_1, 'P', self.BP['p_2'], self.fluid)
        self.h_2 = self.h_1 + (h_2_is - self.h_1) / self.eta_is
        self.T_2 = PropsSI('T', 'P', self.BP['p_2'], 'H', self.h_2, self.fluid)
        self.m_dot = self.lbd * self.V_dot_th /3600 / v_1 
        self.P_i = self.m_dot * (self.h_2 - self.h_1) #innere Leistung
        self.P_el = self.P_i / self.eta_el #elektrische Leistung

In [12]:
V1 = Verdichter(fluid = 'CO2', Fördervolumen = 15, isentroper_Guetegrad = 0.7, elektrischer_Wirkungsgrad = 0.95,
                Liefergrad = 0.8,                 
                Betriebsparameter = BP1)

In [13]:
V1.berechnen()
print('Betriebsparameter:', V1.BP)
print('P_i:', V1.P_i)

Betriebsparameter: {'p_1': 4500000.0, 'T_1': 293.15, 'p_2': 9000000.0, 'n': 50}
P_i: 16087.820158068509


In [14]:
V1.update_BP({'n': 30})
V1.berechnen()
print('Betriebsparameter:', V1.BP)
print('P_i:', V1.P_i)
print('m_dot:', V1.m_dot)

Betriebsparameter: {'p_1': 4500000.0, 'T_1': 293.15, 'p_2': 9000000.0, 'n': 30}
P_i: 9652.692094841106
m_dot: 0.23376749190942214


### Zwei Verdichter in Boosterschaltung

#### Annahmen:
+ Kältemittel R744
+ Saugdruckzustand des Verdichters 1: 20 bar, -10°C
+ Verdichtungsenddruck des Verdichters 2: 90 bar
+ Zwischenkühlung auf 30°C

#### Fragen
+ Wie groß ist der Mitteldruck? Der Massenstrom?

In [15]:
p1 = 20e5
T1 = -10 + 273.15
p3 = 90e5
V1 = Verdichter(fluid = 'CO2', Fördervolumen = 15, isentroper_Guetegrad = 0.7, elektrischer_Wirkungsgrad = 0.95,
                Liefergrad = 0.8)
V2 = Verdichter(fluid = 'CO2', Fördervolumen = 5, isentroper_Guetegrad = 0.7, elektrischer_Wirkungsgrad = 0.95,
                Liefergrad = 0.8)

In [16]:
def Massenbilanz(p2, T2, n_V1, n_V2):
    V1.update_BP({
            'p_1' : p1, # Eintrittsdruck
            'T_1' : T1, # Eintrittstemperatur
            'p_2' : np.asscalar(p2), # Austrittsdruck
            'n' : n_V1, # Drehzahl
        })
    V2.update_BP({
            'p_1' : np.asscalar(p2), # Eintrittsdruck
            'T_1' : T2, # Eintrittstemperatur
            'p_2' : p3, # Austrittsdruck
            'n' : n_V2, # Drehzahl
        })
    V1.berechnen()
    V2.berechnen()
    return V1.m_dot - V2.m_dot

In [17]:
res = opt.root(fun = Massenbilanz,
            x0 = 40e5, # Startwert für p2
            args=(
                20+273.15, # T nach Zwischenkühler
                40, # Drehzahl Verdichter 1
                50, # Drehzahl Verdichter 2
            ),
            method='hybr'
        )
print(res)

    fjac: array([[-1.]])
     fun: 1.1074474670635936e-14
 message: 'The solution converged.'
    nfev: 7
     qtf: array([1.47402124e-09])
       r: array([4.71778134e-08])
  status: 1
 success: True
       x: array([4503580.80660254])


C:\Users\ASUS\AppData\Local\Temp/ipykernel_11132/234195462.py:5: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'p_2' : np.asscalar(p2), # Austrittsdruck
C:\Users\ASUS\AppData\Local\Temp/ipykernel_11132/234195462.py:9: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'p_1' : np.asscalar(p2), # Eintrittsdruck


#### Überprüfen der Ergebnisse

In [18]:
print(V1.BP, V2.BP)
print(V1.m_dot, V2.m_dot)

{'p_1': 2000000.0, 'T_1': 263.15, 'p_2': 4503580.806602544, 'n': 40} {'p_1': 4503580.806602544, 'T_1': 293.15, 'p_2': 9000000.0, 'n': 50}
0.13003964437898646 0.1300396443789754
